In [1]:
import numpy as np
import pandas as pd
import re
from os import listdir
from sklearn.linear_model import MultiTaskElasticNetCV

In [2]:
#SET PATH
#Input takes 'locally' or 'remotely' and returns an absolute path based on the input 
location=input('Are you working locally or remotely today?')

if location=='locally':
    path="/Users/cameronkelsey/Documents/smack_lab/cayo_data/"
elif location=='remotely':
    path="/scratch/ckelsey4/Cayo_meth/"

In [3]:
#Import metadata
metadata=pd.read_table(path + "blood_metadata_full.txt", sep=r'\s+')

#Import M/Cov
regions_cov=pd.read_table(path + "regions_cov_filtered.txt", sep=r'\s+')
regions_cov=regions_cov[regions_cov.columns.intersection(metadata['lid_pid'])]
regions_cov.index=regions_cov.index.str.split('.').str[3]

regions_m=pd.read_table(path + "regions_m_filtered.txt", sep=r'\s+')
regions_m=regions_m[regions_m.columns.intersection(regions_cov.columns)]
regions_m.index=regions_m.index.str.split('.').str[3]

#Match metadata columns to M and Cov matrices
metadata=metadata[metadata['lid_pid'].isin(regions_cov.columns)]

In [5]:
#Define natural sorting function
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]

In [6]:
#Import coverage data per chromosome
dnam_path=path + "dnam_clock/"

cov_files=[f for f in listdir(dnam_path)
        if "cov" in f]

cov_files=sorted(cov_files, key=natural_key)        

cov_full=[]

for file in cov_files:
        file=dnam_path + file        
        dd=pd.read_csv(file, sep=" ")
        cov_full.append(dd)

#Remove last instance of dd to free up memory
del dd

In [10]:
#Import count data per chromosome
m_files=[f for f in listdir(dnam_path)
        if "m" in f]

m_files=sorted(m_files, key=natural_key)        

m_full=[]

for file in m_files:
        file=dnam_path + file        
        dd=pd.read_csv(file, sep=" ")
        m_full.append(dd)

#Remove last instance of dd to free up memory
del dd

In [7]:
#Check that col names for M, Cov and metadata lid_pid are all equal and generate %methylation matrix
if all(regions_m.columns == regions_cov.columns) == True:
    p_meth=regions_m/regions_cov
    p_meth=p_meth.fillna(0)
    print("M and Cov columns match. %methylation matrix generated. NAs replaced with 0's. Onwards!")


M and Cov columns match. %methylation matrix generated. NAs replaced with 0's. Onwards!


In [8]:
#Remove X-chromosome from p_meth
#Generate list of chrs where n=n of regions
chrs=[i[0] for i in p_meth.index.str.split('_', n=1)]
p_meth['chr']=chrs

#Filter out X-chrom
p_meth=p_meth[p_meth['chr'] != 'X']

#Print list of reminaing chrs to be sure
print(p_meth['chr'].unique())
print("Values should be between 1 and 20")

#Remove chr col from p_meth
p_meth=p_meth.drop('chr', axis=1)

#Test whether chr column was dropped from p_meth
len(p_meth.columns) == len(regions_cov.columns)

#Transpose df so cols are regions and rows are lids
p_meth=p_meth.transpose()

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20']
Values should be between 1 and 20


In [36]:
#Actual
SAMP=4

p_meth_short=p_meth.iloc[:, 0:5]

#Define train and test sets for methylation
train_epi=p_meth_short.drop(p_meth.index[SAMP], axis=0)
test_epi=p_meth_short.iloc[SAMP:SAMP+1, :]

#Define train and test set for age
train_age=metadata.drop(metadata.index[SAMP], axis=0).iloc[:, 20:21]
test_age=metadata.iloc[SAMP:SAMP+1, 20:21]

#Define alphas to test
alphas=np.arange(0.1, 1.1, 0.1)


In [37]:

model=MultiTaskElasticNetCV(l1_ratio=alphas, random_state=0)

In [38]:
fitted_model=model.fit(train_epi, train_age)

In [39]:
fitted_model.predict(test_epi)

array([[11.69907552]])

In [43]:
print(fitted_model.alpha_)
print(fitted_model.l1_ratio_)
mse_vals=fitted_model.mse_path_
mse_vals.flatten()


0.016404368446576046
0.9


array([36.69088066, 29.88568476, 31.42829381, ..., 30.44942337,
       36.18593258, 44.66792482], shape=(5000,))